# CPSC 536M - Homework 2
Emma Hansen and Naomi Graham

October 2020

In [ ]:
# Load packages
using Images, FileIO, ImageMagick, Zygote, Random, LinearAlgebra, Colors, DSP,ToeplitzMatrices, SparseArrays, Wavelets, LazySets
using PyPlot, FFTW, ForwardDiff

In [ ]:
# Frank-Wolfe
function frank_wolfe(tau,f,f_grad,K,x0_wav,wav)
    # x0 is a vector
    n = size(x0_wav)[1]
    
    for k=1:K
        x0 = idwt(x0_wav, wav, 3)
        grad_f = f_grad(x0)
        ind_max = argmax(grad_f)
        s = -tau * sign(grad_f[ind_max])* I(n)[:,ind_max] # from http://www.stat.cmu.edu/~ryantibs/convexopt-S15/scribes/23-cond-grad-scribed.pdf
        
        gamma = 2 / (2+ sqrt(k))
        x1_wav = ((1-gamma) .* x0_wav) + (gamma .* s)
        x0_wav = deepcopy(x1_wav)
        #x_history[:,k+1] = x0
        
        if mod(k,100)==0
            print("k is: ")
            println(k)
            print("the function value is: ")
            println(f(idwt(x0_wav, wav, 3)))
        end
    end
    return x0_wav
end

In [ ]:
function ConvolutionMatrix(Kernel,N)
    # N is length of image to be convolved with (must be a square image)
    toeplitzrow = hcat(Kernel[1,:]',zeros(1,N-size(Kernel)[1]))
    for i=2:size(Kernel)[1]
        toeplitzrow = hcat(toeplitzrow,Kernel[i,:]',zeros(1,N-size(Kernel)[1]))
    end
    l = size(toeplitzrow)[2]
    toeplitzrow = hcat(toeplitzrow,zeros(1,N^2-l))

    toeplitz = Circulant(toeplitzrow')'
    
    return sparse(toeplitz)
end

In [ ]:
# define a function to calculate the objective value

function objective(X_vec,R,B_vec)
# function objective(X_vec,R,B_vec)
# X = idwt(X_wav, wav, 3) # commented out since derivative doesn't like the transform to occur in the objective...
    return (1/2) * sum(( (R * X_vec) - B_vec) .^ 2)
end

##
#function obejctive_gradient(X_vec,R,B_vec,wav)
function obejctive_gradient(X_vec,R,B_vec,wav)
    #gradfx = gradient(y -> objective(y,R,B_vec) , X_vec)
    #return dwt(gradfx[1], wav, 3)
    return dwt((R' * R * X_vec) - (R' * B_vec), wav, 3)
end

## Loading images

In [ ]:
# Clear image
original_path = "smaller_climber128.jpg"
original = load(original_path)
original_grey = Gray.(original)
original_array = convert(Array{Float64}, original_grey)
n, = size(original_array)

In [ ]:
# Blurring in photoshop
blurry_path = "smaller_climber_blurry.jpg"
blurry1 = load(blurry_path)
blurry_grey = Gray.(blurry1)
blurry1_array = convert(Array{Float64}, blurry_grey)

In [ ]:
# Blurring in Julia
Kernel = (1/256) .* [1 4 6 4 1; 4 16 24 16 4; 6 24 36 24 6; 4 16 24 16 4; 1 4 6 4 1]
m = size(Kernel)[1]
conv_matrix = ConvolutionMatrix(Kernel,n)
blurry_array = reshape(conv_matrix * original_array[:],size(original_array))


mosaicview(original, blurry1, Gray.(blurry_array); nrow=1)

## Set parameters and initial values

In [ ]:
#K = 5001 # frank-wolfe iterations

K = 10001 # frank-wolfe iterations
wt = wavelet(WT.haar) # define wavelet to use in transform
wt = WT.scale(wt, 1) # 1/sqrt(2) scales output to be between 0 and 1 - useful? idk


# define B
BLURRY = blurry_array

# take wave transform of blurry image for starting point
X0_wav = dwt(BLURRY,wt,3)

# parameter 
tau = 1800 # make tau > the 1-norm of the wavelet transform of blurry matrix


tau = 2800 # make tau > the 1-norm of the wavelet transform of blurry matrix



## Define functions with fixed parameters

In [ ]:
# defining objective for input data
f = (y -> objective(y,conv_matrix,BLURRY[:]))

# defining gradient for input data
f_grad = (y -> obejctive_gradient(y,conv_matrix,BLURRY[:],wt))

## Run the algorithm!

In [ ]:
X_wav_sol = frank_wolfe(tau,f,f_grad,K,X0_wav[:],wt); # call frank-wolfe
X_sol = idwt(X_wav_sol, wt, 3); # inverse wavelet transform to bring back to the image domain

In [ ]:
X = reshape(X_sol,size(blurry_array))
Gray.(X)

### Frank-Wolfe Algorithm

### Print Result

In [ ]:
# defining line search function

function linesearch(grad_f,s,d)
    #alpha_upper0 = 1
    #alpha_lower0 = 0
    alpha_mid = 0.5
    alpha_range = 0.5
    stop = 50
    alpha_storage = zeros(stop+1)
    alpha_storage[1] = alpha_mid
    
    for i=1:stop
        val_mid = dot(-grad_f, alpha_mid * s + (1-alpha_mid) * d) / (norm(grad_f)*norm(alpha_mid * s + (1-alpha_mid) * d))
        alpha_upper = alpha_mid + (1/2)*alpha_range
        alpha_lower = alpha_mid - (1/2)*alpha_range
        
        val_upper = dot(-grad_f, alpha_upper * s + (1-alpha_upper) * d) / (norm(grad_f)*norm(alpha_upper * s + (1-alpha_upper) * d))
        val_lower = dot(-grad_f, alpha_lower * s + (1-alpha_lower) * d) / (norm(grad_f)*norm(alpha_lower * s + (1-alpha_lower) * d))
        
        if val_upper > val_mid # note: I think something is wrong with this criteria...
            alpha_mid = deepcopy(alpha_upper)
        elseif val_lower > val_mid
            alpha_mid = deepcopy(alpha_lower)
        end
        
        alpha_range = (1/2) * alpha_range
        alpha_storage[i+1] = deepcopy(alpha_mid)
    end
    return alpha_mid, alpha_storage
end



In [ ]:
# Load packages
using Images, FileIO, ImageMagick, Zygote, Random, LinearAlgebra, Colors, DSP,ToeplitzMatrices, SparseArrays, Wavelets, LazySets
using PyPlot, FFTW, ForwardDiff

In [ ]:
wt = wavelet(WT.haar) # define wavelet to use in transform
wt = WT.scale(wt, 1) # 1/sqrt(2) scales output to be between 0 and 1 - useful? idk

function normsquared(x)
    return sum(x .^ 2)
end

In [ ]:
# Clear image
original_path = "smaller_climber128.jpg"
original = load(original_path)
original_grey = Gray.(original)
original_array = convert(Array{Float64}, original_grey)

n, = size(original_array)

In [ ]:
# Blurring in photoshop
blurry_path = "smaller_climber_blurry.jpg"
blurry1 = load(blurry_path)
blurry_grey = Gray.(blurry1)
blurry1_array = convert(Array{Float64}, blurry_grey)

In [ ]:
# Blurring in Julia
#Kernel = (1/16) .* [1 2 1;2 4 2;1 2 1] # gaussian blur 3x3 from https://en.wikipedia.org/wiki/Kernel_(image_processing)
Kernel = (1/256) .* [1 4 6 4 1; 4 16 24 16 4; 6 24 36 24 6; 4 16 24 16 4; 1 4 6 4 1]
m = size(Kernel)[1]
conv_matrix = ConvolutionMatrix(Kernel,n)
blurry_array = reshape(conv_matrix * original_array[:],size(original_array))

In [ ]:
mosaicview(original, blurry1, Gray.(blurry_array); nrow=1)

In [ ]:
Gray.(blurry_array)

In [ ]:
Gray.(original_array)

In [ ]:
println(original_array)
println(blurry_array)

In [ ]:
org_wav = dwt(original_array,wt,3)
Gray.(org_wav)


In [ ]:
inv_orgwav = idwt(org_wav,wt,3)
Gray.(inv_orgwav)

In [ ]:
blur_wav = dwt(blurry_array,wt,3)
Gray.(blur_wav)

In [ ]:
conv_wav = 0.5.*blur_wav + 0.5.*org_wav
Gray.(conv_wav)

In [ ]:
inv_conv_wav = idwt(conv_wav,wt,3)
println(inv_conv_wav)

In [ ]:
Gray.(inv_conv_wav)


In [ ]:
sum(abs.(org_wav))

In [ ]:
sum(abs.(blur_wav))